In [4]:
!pip install einops transformers==4.29.2 peft omegaconf torch accelerate --user




[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Wifi1/DNABERT2",trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained("Wifi1/DNABERT2",trust_remote_code=True)

C:\Users\shash/.cache\huggingface\modules\transformers_modules\zhihan1996\DNABERT-2-117M\25abaf0bd247444fcfa837109f12088114898d98\bert_layers.py:125: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


In [6]:
import torch


In [7]:
sequence = "CCAAGTTTCTTTTGCACATGTCGTAAATAGCCTCATTATCGACCATGAAAGTACAATCTGCATGTTCCAATGTAGTATGTGTAGTTAAAACTGTGTTGTAAGGCTCAACCACAGAAGTAGACACTTGTGGAGCAGGGTATACGGCAAATTCCAGCTTGGATTTCTTACCGTATTCAGCGGATAGTTCTTCCAACAGTAAGGACCCTAGACCGGAACCAGTACCACCACCAAGAGAATGGGTAAACAAGAACCCTTGTAACCCATCACATTGGTCTGCCAGTTTTCTAATCCTATCCAGAACATCGCCCAAAATTTCTCTACCAACGGTGTAATGGCCTCTTGCATAATTATTAGCCGCGTCCTCCTTACCACTGATCAACTGTTCTGGATGGAACAAGTCCTTGTAAGGACCGTTACGGACTTCGTCAATAACGTTGGGCTCTAAATCCACGTAAATAGCCCTTGGAACGAACTTACCGTAGCCGGTTTCATGGAAAAAC"

# Tokenize the input sequence
encoded_input = tokenizer(sequence, return_tensors='pt')
# Pass the encoded input through the model
with torch.no_grad():
    outputs = model(input_ids=encoded_input['input_ids'], attention_mask=encoded_input['attention_mask'])
    logits = outputs[0]
    predicted_class = logits.argmax(-1).item()
    print(f"Predicted class: {predicted_class}")


Predicted class: 1


In [8]:
import pandas as pd
import numpy as np


In [9]:
data = pd.read_csv('D:\\College Files\\Project School\\Main Project\\Datasets\\test1.csv')

In [10]:
x_test=data["sequence"].to_numpy()
y_test=data["label"].to_numpy()
arr = np.array([])
y_test.shape


(1850,)

In [11]:
from sklearn.metrics import accuracy_score

In [12]:
from tqdm import tqdm  # Import tqdm for progress tracking

predictions = []  # Store predictions in a separate list

# Wrap the loop with tqdm for progress tracking
for sequence in tqdm(x_test, desc='Predicting'):
    encoded_input = tokenizer(sequence, return_tensors='pt')

    # Pass the encoded input through the model
    with torch.no_grad():
        outputs = model(input_ids=encoded_input['input_ids'], attention_mask=encoded_input['attention_mask'])
        logits = outputs[0]
        predicted_class = logits.argmax(-1).item()

    predictions.append(predicted_class)  # Append prediction to the `predictions` list


Predicting: 100%|██████████████████████████████████████████████████████████████████| 1850/1850 [06:27<00:00,  4.78it/s]


In [13]:

arr=np.array(predictions)
arr.shape

(1850,)

In [14]:
accuracy = accuracy_score(y_test, arr)
print("Accuracy:", accuracy*100)

Accuracy: 94.48648648648648


In [15]:
model


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_feat